In [56]:
def Generate_Merfish_Code(num_bits, on_bits, hamming_distance, 
                          num_regs=None, randomize=True, repeat=500, verbose=True):
    '''Function to generate a MERFISH encoding scheme with given number of bits, on-bits and hemming distance
    Inputs:
        num_bits: total number of bits for MERFISH code, int
        on_bits: number of on-bits for this code, int (< num_bits)
        hemming_distance: minimum hemming distance allowed for this code, int (< num_bits)
        num_regs: number of different regions, if -1 then generate all, int (default: -1)
        randomize: whether randomly choose from candidate codes, bool (default: True)
        repeat: number of repeats in generating codes, int (default: 100)
        verbose: say something!, bool (default: True)
    Output:
        hyb_matrix: encoding scheme, num_bits by num_regs 0-1 array
        '''
    import numpy as np
    from itertools import combinations
    from random import randint
    from time import time
    _start = time()
    # convert inputs into int
    _num_bits = np.int(num_bits)
    _on_bits = np.int(on_bits)
    _d = np.int(hamming_distance)
    if verbose:
        print("- Generate encoding scheme of "+str(_num_bits)+" bits with "+str(_on_bits)+" on-bits")
        print("-- hamming distance is", _d)
    # check inputs
    if _num_bits < _on_bits:
        raise ValueError('on-bits is larger than total_bits!')
    if _num_bits < _d:
        raise ValueError('hamming_distance is larger than total_bits!')
    if num_regs == None:
        num_regs = np.inf

    def code_distance(_code1, _code2):
        '''given two codes, calculate hamming distance'''
        _c1 = list(_code1)
        _c2 = list(_code2)
        _overlap = 0
        for _on_bit in _c1:
            if _on_bit in _c2:
                _overlap += 1
        _distance = len(_c1) + len(_c2) - 2 * _overlap
        #each on unique on bit is 1 distance, then remove overlapped on bits (count *2 from each code)
        return _distance
    # number of repeats
    _repeat_num = 1 + int(randomize) * repeat
    if verbose:
        print("-- number of repeats:", _repeat_num)
    _best_coding, _best_var = [], np.inf
    for _i in range(_repeat_num):
        # generate all code as candidates
        _cand_codes = list(combinations(list(range(_num_bits)), _on_bits))
        # initialize chosen codes
        _chosen_codes = []
        _chosen_codes.append(_cand_codes.pop(0))
        # loop through other candidate codes, find all good code
        while len(_cand_codes) > 0 or len(_chosen_codes) >= num_regs:
            if randomize: # randomly loop through the remaining cand codes
                _rand_id = randint(0, len(_cand_codes)-1)
                _c = _cand_codes.pop(_rand_id)
            else: # loop through from the first remaining cand codes
                _c = _cand_codes.pop(0)
            _keep = True
            # calculate hamming dist between the new candidate _c with all other already chosen code
            for _chosen_c in _chosen_codes:
                if code_distance(_c, _chosen_c) < _d:
                    _keep = False
                    break
            if _keep:
                _chosen_codes.append(list(_c))
        _uid, _ucount = np.unique(np.array(_chosen_codes), return_counts=True)
        # _uid: unique bit appeared 0-15 (e.g., from [0,1,2,3],[1,2,10,11],[1,3,7,9], etc...)
        # _ucount: how many times each unique bit is used
        
        # find as many as chosen codes (with valid hamming distance) with as uniform usage of each bit as possible
        if len(_chosen_codes) > len(_best_coding) and np.var(_ucount) <= _best_var:
            _best_coding = _chosen_codes
            _best_var = np.var(_ucount)
            if verbose:
                print("-- length of all possible code", len(_best_coding), ", variance", _best_var)
    # select subset
    if not num_regs:
        _select_set = _best_coding
    else: # select subset
        _select_set, _select_var = [], np.inf
        
    
    
    _end = time()
    if verbose:
        print("-- Duration: ", _end-_start)
        
    
    return _best_coding



if __name__ =='__main__':
    _best_coding = Generate_Merfish_Code(16, 4, 4, randomize=True)
    
    

- Generate encoding scheme of 16 bits with 4 on-bits
-- hamming distance is 4
-- number of repeats: 501
-- length of all possible code 101 , variance 1.4375
-- length of all possible code 104 , variance 1.0
-- length of all possible code 106 , variance 0.875
-- length of all possible code 107 , variance 0.6875
-- Duration:  35.62431716918945


In [60]:
len(_best_coding)

#_best_coding

107

In [58]:
_uid, _ucount = np.unique(np.array(_best_coding), return_counts=True)

_uid


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [63]:
print(_ucount)

np.var(_ucount)

[28 27 27 28 27 27 26 28 26 26 26 26 28 26 26 26]


0.6875

In [65]:
_ucount_test = [38, 37, 27, 28, 27, 27, 26, 28, 26, 26, 26, 26, 28, 26, 16, 16]

np.var(_ucount_test)

29.4375